# Lecture 9: Ensemble Learning

## Exercise 1: Implement Bagging Aggregation

**Task**: Write a Python function bagging_aggregation that takes a list of predictions from multiple models for a classification task and outputs the final aggregated prediction using majority voting.

- Input: A list of lists, where each inner list contains predictions from a single model.

- Output: A list of final predictions (aggregated using majority voting).

**Example**:

```python
predictions = [[1, 0, 1], [1, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 1]] # predictions from 5 models for 3 samples
output = [0, 0, 1]
```

In [ ]:
from collections import Counter

def bagging_aggregation(predictions):
    aggregated = []
    for i in range(len(predictions[0])):
        column = [pred[i] for pred in predictions]
        aggregated.append(Counter(column).most_common(1)[0][0])
    return aggregated


In [ ]:
import unittest

class TestBaggingAggregation(unittest.TestCase):
    def test_majority_voting(self):
        self.assertEqual(bagging_aggregation([[1, 0, 1], [1, 1, 1], [0, 0, 1]]), [1, 0, 1])
    
    def test_tie_handling(self):
        self.assertEqual(bagging_aggregation([[1, 0], [0, 1], [1, 0]]), [1, 0])  # Majority voting resolves ties arbitrarily.
    
    def test_single_model(self):
        self.assertEqual(bagging_aggregation([[1, 1, 1]]), [1, 1, 1])

if __name__ == "__main__":
    unittest.main()

## Exercise 2: Boosting Weight Update

**Task**: Write a Python function update_weights that updates weights for a boosting algorithm. Given a list of current weights, a list of binary predictions, and the actual labels, the function adjusts the weights by increasing the weight of misclassified samples.

- Input: A list of current weights, a list of predictions, and a list of true labels.

- Output: A list of updated weights.

**Example**:

```python
# Input: 
weights=[0.2, 0.3, 0.5]
predictions=[1, 0, 1]
labels=[1, 1, 0]

# Output: 
output = [0.2, 0.6, 1.0]  # (weights are normalized)
```

In [ ]:
def update_weights(weights, predictions, labels):
    updated_weights = [
        weight * 2 if pred != label else weight
        for weight, pred, label in zip(weights, predictions, labels)
    ]
    total_weight = sum(updated_weights)
    return [w / total_weight for w in updated_weights]

In [ ]:
class TestUpdateWeights(unittest.TestCase):
    def test_weight_update(self):
        self.assertEqual(update_weights([0.2, 0.3, 0.5], [1, 0, 1], [1, 1, 0]), [0.2, 0.6, 1.0])
    
    def test_all_correct_predictions(self):
        self.assertEqual(update_weights([0.2, 0.3, 0.5], [1, 1, 0], [1, 1, 0]), [0.2, 0.3, 0.5])
    
    def test_all_incorrect_predictions(self):
        self.assertEqual(update_weights([0.2, 0.3, 0.5], [0, 0, 1], [1, 1, 0]), [0.25, 0.375, 0.625])

if __name__ == "__main__":
    unittest.main()

## Exercise 3: Stacking Meta-Model Predictions

**Task**: Write a Python function stacking_predictions that takes the predictions of multiple models (as a 2D list) and a meta-model (as a callable function) to predict the final output.

- Input: A 2D list of predictions (where each row is from a model) and a callable meta-model (e.g., a lambda function).

- Output: A list of final predictions from the meta-model.

**Example**:

```python
# Input: 
predictions=[[0.8, 0.1], [0.6, 0.4], [0.7, 0.2]]
meta_model=lambda x: 1 if sum(x)/len(x) > 0.5 else 0

# Output: 
output = [1, 0]
```

In [ ]:
def stacking_predictions(predictions, meta_model):
    transposed = list(zip(*predictions))
    return [meta_model(row) for row in transposed]

In [ ]:
class TestStackingPredictions(unittest.TestCase):
    def test_average_meta_model(self):
        meta_model = lambda x: 1 if sum(x) / len(x) > 0.5 else 0
        self.assertEqual(stacking_predictions([[0.8, 0.1], [0.6, 0.4], [0.7, 0.2]], meta_model), [1, 0])
    
    def test_majority_vote_meta_model(self):
        meta_model = lambda x: 1 if sum(x) > len(x) / 2 else 0
        self.assertEqual(stacking_predictions([[1, 0], [1, 1], [0, 0]], meta_model), [1, 0])
    
    def test_single_model(self):
        meta_model = lambda x: x[0]
        self.assertEqual(stacking_predictions([[1, 0]], meta_model), [1, 0])

if __name__ == "__main__":
    unittest.main()